In [ ]:
import pandas as pd
from apputil import load_processed, train_baseline

df = load_processed()
model, feature_cols, auc = train_baseline(df)
auc

In [ ]:
# Smoke-test prediction

from apputil import predict_gross

a = "The Dark Knight"
b = "Inception"
predict_gross(a, b, df, model, feature_cols)

In [ ]:
# Save artifacts
# able to reuse the trained model without retraining every run:

import joblib, os
os.makedirs("models", exist_ok=True)
joblib.dump({"model": model, "feature_cols": feature_cols}, "models/baseline_logreg.joblib")